In [ ]:
import yfinance as yf
import os
import pandas as pd

# Create a directory to store S&P 500 stock data
directory = "SP500_10_25"
if not os.path.exists(directory):
    os.makedirs(directory)

# Retrieve the list of S&P 500 stocks from Wikipedia
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(sp500_url)[0]
tickers = table['Symbol'].tolist()

# Add VIX and S&P 500 index (^VIX, ^GSPC)
tickers.extend(["^VIX", "^GSPC"])

# Fetch stock data and save as CSV
for ticker in tickers:
    print(f"Fetching data for: {ticker}")
    stock_data = yf.download(ticker, start="2010-01-01", end="2025-04-01")
    
    # Check if the number of data points is less than 100 days
    if len(stock_data) < 100:
        print(f"Skipping {ticker}, insufficient data: {len(stock_data)} days")
        continue  # Skip this stock and do not save data

    # Keep only the required columns (Open, High, Low, Close, Volume)
    stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # Save data as CSV
    stock_data.to_csv(f"{directory}/{ticker}.csv")
    print(f"Data for {ticker} has been saved")

print("All eligible stock data has been successfully saved.")


In [ ]:
import os
import pandas as pd

# Set storage path
directory = "SP500_10_25"

# Process each CSV file
for ticker in os.listdir(directory):
    if ticker.endswith(".csv"):
        file_path = os.path.join(directory, ticker)
        
        # Read the CSV file
        df = pd.read_csv(file_path)

        # Delete the second and third rows
        df = df.drop([0, 1]).reset_index(drop=True)

        # Remove rows containing NaN values
        df = df.dropna()

        # Rename the first column to 'Date'
        df.columns.values[0] = 'Date'

        # Save the modified file
        df.to_csv(file_path, index=False)

        print(f"Processing complete: {ticker}")

print("🎉 All CSV files have been cleaned!")


In [3]:
import os
import pandas as pd

directory = "SP500_10_25"

# Get a list of all CSV files
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# Initialize an empty list to store individual stock DataFrames
dfs = []

# Read and store each CSV file
for file in csv_files:
    stock_df = pd.read_csv(os.path.join(directory, file), parse_dates=["Date"])
    stock_df["Stock_ID"] = file.replace(".csv", "")  # Add stock ticker as an identifier
    dfs.append(stock_df)

# Concatenate all stock DataFrames
training_df = pd.concat(dfs, ignore_index=True)

# Sort by Date first, then by Stock_ID
training_df = training_df.sort_values(by=["Date", "Stock_ID"]).reset_index(drop=True)

# Reorder columns: Date → Stock_ID → Other Columns
cols = ["Date", "Stock_ID"] + [col for col in training_df.columns if col not in ["Date", "Stock_ID"]]
training_df = training_df[cols]

# Create new directory
directory = "training"
if not os.path.exists(directory):
    os.makedirs(directory)

# Save merged data to a new CSV file
training_csv_path = os.path.join(directory, "10_25_merged_stocks.csv")
training_df.to_csv(training_csv_path, index=False)

print(f"Merging completed. Merged file saved as '{training_csv_path}'.")

training_df.head(10)

Merging completed. Merged file saved as 'training/10_25_merged_stocks.csv'.


,Date,Stock_ID,Open,High,Low,Close,Volume
0,2010-01-04,A,20.073585,20.227063,19.907317,20.016031,3815561
1,2010-01-04,AAPL,6.422875,6.455075,6.391277,6.440330,493729600
2,2010-01-04,ABT,18.573218,18.700033,18.480677,18.665758,10829095
3,2010-01-04,ACGL,7.587114,7.628319,7.580774,7.601905,4813200
4,2010-01-04,ACN,31.524212,32.040505,31.509027,31.941801,3650100
5,2010-01-04,ADBE,36.650002,37.299999,36.650002,37.090000,4710200
6,2010-01-04,ADI,22.246505,22.526422,22.120542,22.162529,2102700
7,2010-01-04,ADM,21.011575,21.251860,20.911456,21.004900,3472500
8,2010-01-04,ADP,26.597105,26.597105,26.083978,26.163391,3930120
9,2010-01-04,ADSK,25.610001,25.830000,25.610001,25.670000,2228600


In [4]:
import pandas as pd
import numpy as np

# Load the merged dataset
file_path = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/10_25_merged_stocks.csv"
df = pd.read_csv(file_path, parse_dates=["Date"])

# Ensure sorting by Date and Stock_ID
df = df.sort_values(by=["Stock_ID", "Date"]).reset_index(drop=True)

# Function to calculate technical indicators
def calculate_features(df):
    df["Return_1d"] = df.groupby("Stock_ID")['Close'].pct_change(1)
    df["Return_5d"] = df.groupby("Stock_ID")['Close'].pct_change(5)
    df["Return_10d"] = df.groupby("Stock_ID")['Close'].pct_change(10)
    df["Return_50d"] = df.groupby("Stock_ID")['Close'].pct_change(50)
    
    # Rolling volatility
    df["Volatility_5d"] = df.groupby("Stock_ID")["Return_1d"].rolling(5).std().reset_index(level=0, drop=True)
    df["Volatility_10d"] = df.groupby("Stock_ID")["Return_1d"].rolling(10).std().reset_index(level=0, drop=True)
    df["Volatility_20d"] = df.groupby("Stock_ID")["Return_1d"].rolling(20).std().reset_index(level=0, drop=True)
    
    # Momentum indicators
    df["SMA_10"] = df.groupby("Stock_ID")["Close"].rolling(10).mean().reset_index(level=0, drop=True)
    df["SMA_50"] = df.groupby("Stock_ID")["Close"].rolling(50).mean().reset_index(level=0, drop=True)
    df["SMA_200"] = df.groupby("Stock_ID")["Close"].rolling(200).mean().reset_index(level=0, drop=True)
    df["RSI_14"] = 100 - (100 / (1 + df.groupby("Stock_ID")["Return_1d"].rolling(14).apply(lambda x: np.mean(x[x > 0]) / np.mean(-x[x < 0]) if np.mean(-x[x < 0]) != 0 else np.inf).reset_index(level=0, drop=True)))
    
    # Volume-based features
    df["Volume_Change_5d"] = df.groupby("Stock_ID")["Volume"].pct_change(5)
    df["Volume_Change_10d"] = df.groupby("Stock_ID")["Volume"].pct_change(10)
    
    return df

# Apply feature calculations
df = calculate_features(df)

# Save the new dataset with features
output_path = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/10_25_merged_stocks_features.csv"
df.to_csv(output_path, index=False)
print(f"Feature engineering complete. Saved to {output_path}")

df.head(10)

Feature engineering complete. Saved to /home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/10_25_merged_stocks_features.csv


,Date,Stock_ID,Open,High,Low,Close,Volume,Return_1d,Return_5d,Return_10d,Return_50d,Volatility_5d,Volatility_10d,Volatility_20d,SMA_10,SMA_50,SMA_200,RSI_14,Volume_Change_5d,Volume_Change_10d
0,2010-01-04,A,20.073585,20.227063,19.907317,20.016031,3815561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-05,A,19.958470,19.964865,19.670698,19.798597,4186031,-0.010863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-06,A,19.728256,19.824180,19.670701,19.728256,3243779,-0.003553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-07,A,19.683500,19.709081,19.504443,19.702686,3095172,-0.001296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-08,A,19.593967,19.728261,19.440491,19.696285,3733918,-0.000325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010-01-11,A,19.747450,19.856164,19.613158,19.709082,4781579,0.000650,-0.015335,NaN,NaN,0.004623,NaN,NaN,NaN,NaN,NaN,NaN,0.253178,NaN
6,2010-01-12,A,19.542811,19.600365,19.325383,19.472467,2871073,-0.012005,-0.016472,NaN,NaN,0.005107,NaN,NaN,NaN,NaN,NaN,NaN,-0.314130,NaN
7,2010-01-13,A,19.485254,19.683496,19.216669,19.625942,3418949,0.007882,-0.005186,NaN,NaN,0.007129,NaN,NaN,NaN,NaN,NaN,NaN,0.054002,NaN
8,2010-01-14,A,19.561993,19.977660,19.504438,19.920107,6163782,0.014989,0.011035,NaN,NaN,0.010080,NaN,NaN,NaN,NaN,NaN,NaN,0.991418,NaN
9,2010-01-15,A,19.964873,20.054402,19.395725,19.459675,4626681,-0.023114,-0.012013,NaN,NaN,0.015319,NaN,NaN,19.712913,NaN,NaN,NaN,0.239096,NaN


In [ ]:
# Based on my previous experience, this is basically the most troublesome ones
print(df[["Volume_Change_5d", "Volume_Change_10d"]].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]))

       Volume_Change_5d  Volume_Change_10d
count      1.818932e+06       1.816425e+06
mean                inf                inf
std                 NaN                NaN
min       -1.000000e+00      -1.000000e+00
1%        -7.373207e-01      -7.489641e-01
25%       -2.729058e-01      -2.878459e-01
50%       -9.225877e-03      -7.963247e-03
75%        3.626752e-01       3.914881e-01
99%        3.062821e+00       3.269729e+00
max                 inf                inf


/home/jesse/Projects/myenvs/xgboost/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/jesse/Projects/myenvs/xgboost/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [6]:
# Replace infinite values with NaN (so they can be dropped)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df = df.dropna().reset_index(drop=True)

# Save the cleaned dataset
file_path = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/10_25_merged_stocks_features.csv"
df.to_csv(file_path, index=False)

print(f"Cleaned dataset saved as '{file_path}'.")

df.head(10)

Cleaned dataset saved as '/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/10_25_merged_stocks_features.csv'.


,Date,Stock_ID,Open,High,Low,Close,Volume,Return_1d,Return_5d,Return_10d,Return_50d,Volatility_5d,Volatility_10d,Volatility_20d,SMA_10,SMA_50,SMA_200,RSI_14,Volume_Change_5d,Volume_Change_10d
0,2010-10-18,A,22.056001,22.235057,21.825784,22.164713,2889666,0.005220,0.026963,0.063516,0.206404,0.008607,0.010332,0.016929,21.603243,19.438315,20.035407,57.077583,0.078135,-0.577275
1,2010-10-19,A,21.825789,22.036821,21.461279,21.755445,5580676,-0.018465,0.010095,0.029661,0.140845,0.013740,0.012511,0.017771,21.665913,19.492032,20.044104,55.296023,0.159694,-0.054881
2,2010-10-20,A,21.832178,22.286216,21.800203,22.164713,3486612,0.018812,0.010201,0.060912,0.176510,0.013772,0.012325,0.017565,21.793171,19.558539,20.055934,54.089284,-0.220259,-0.229009
3,2010-10-21,A,22.247852,22.382144,22.011241,22.222271,4276622,0.002597,0.015191,0.046687,0.231396,0.013549,0.011812,0.017337,21.892292,19.642057,20.068404,52.142750,0.050118,-0.181446
4,2010-10-22,A,22.215877,22.330985,22.075189,22.222271,3521422,0.000000,0.007831,0.028107,0.262259,0.013375,0.010873,0.014028,21.953044,19.734399,20.081002,63.914688,-0.094475,-0.120312
5,2010-10-25,A,22.401324,22.484459,22.222266,22.318192,3706098,0.004316,0.006924,0.034074,0.276051,0.013320,0.010772,0.013934,22.026586,19.830962,20.094112,61.602457,0.282535,0.382745
6,2010-10-26,A,22.145535,22.382145,21.998451,22.254248,4473740,-0.002865,0.022928,0.033254,0.281296,0.008411,0.010819,0.013945,22.098209,19.928676,20.106838,67.696063,-0.198352,-0.070333
7,2010-10-27,A,22.049604,22.228661,21.851362,22.119947,2316206,-0.006035,-0.002020,0.008161,0.181353,0.004160,0.009681,0.014083,22.116115,19.996590,20.120075,65.918312,-0.335686,-0.482007
8,2010-10-28,A,22.465268,22.593165,22.011231,22.330975,3349608,0.009540,0.004892,0.020157,0.186141,0.006111,0.009971,0.013282,22.160238,20.066678,20.133600,63.156453,-0.216763,-0.177509
9,2010-10-29,A,22.267037,22.388539,22.139139,22.254248,2703592,-0.003436,0.001439,0.009281,0.219342,0.006435,0.009919,0.013187,22.180702,20.146742,20.145271,61.816279,-0.232244,-0.304778


In [ ]:
# Double check again!
print(df[["Volume_Change_5d", "Volume_Change_10d"]].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]))

       Volume_Change_5d  Volume_Change_10d
count      1.721096e+06       1.721096e+06
mean       1.645839e-01       1.799560e-01
std        2.604063e+00       2.004142e+00
min       -1.000000e+00      -1.000000e+00
1%        -7.347697e-01      -7.460416e-01
25%       -2.720090e-01      -2.872004e-01
50%       -8.942129e-03      -7.647558e-03
75%        3.616978e-01       3.909008e-01
99%        3.005400e+00       3.201664e+00
max        2.828333e+03       2.223667e+03


In [8]:
import pandas as pd

# Load the merged dataset
file_path = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/10_25_merged_stocks_features.csv"
df = pd.read_csv(file_path, parse_dates=["Date"])

# Define split dates
train_end_date = "2020-12-31"
valid_start_date = "2021-01-01"
valid_end_date = "2021-12-31"
test_start_date = "2022-01-01"
test_end_date = "2025-04-01"

# Split into training, validation, and testing sets
train_df = df[df["Date"] <= train_end_date]
valid_df = df[(df["Date"] >= valid_start_date) & (df["Date"] <= valid_end_date)]
test_df = df[(df["Date"] >= test_start_date) & (df["Date"] <= test_end_date)]

# Save to CSV
output_dir = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training"
train_path = f"{output_dir}/train_2010_2020.csv"
valid_path = f"{output_dir}/valid_2021_2021.csv"
test_path = f"{output_dir}/test_2022_2025.csv"

train_df.to_csv(train_path, index=False)
valid_df.to_csv(valid_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Training set saved: {train_path} ({len(train_df)} rows)")
print(f"Validation set saved: {valid_path} ({len(valid_df)} rows)")
print(f"Testing set saved: {test_path} ({len(test_df)} rows)")


Training set saved: /home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/train_2010_2020.csv (1193146 rows)
Validation set saved: /home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/valid_2021_2021.csv (124009 rows)
Testing set saved: /home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/test_2022_2025.csv (403941 rows)
